In [1]:
def collatz(n):
    if n % 2 == 0:
        return n/2
    else:
        return 3*n+1

In [17]:
# n = 93571393692802302
n = 7887663552367

steps = 0
while True:
    n = collatz(n)
    steps += 1
    if n == 1:
        break

print(steps)

511
